<a href="https://colab.research.google.com/github/Lily999-ru/Preddiplomnaya_Praktika/blob/main/smoke_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# БИБЛИОТЕКИ

# Импорт токенизатора и модели RuBERT.
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Для запуска модели.
import torch
import torch.nn.functional as F
import numpy as np
import re

# Установка библиотеки для определения языка вводных данных.
!pip install langdetect

# Определение языка входного текста
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Подключение Google Drive для сохранения данных на облаке
from google.colab import drive
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5bb0cbb3bfd963cd67e7a2c51bf2907a5f42486f2707416fb6e13e5ccf20ea98
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
Mounted at /content/drive


In [6]:
# ЗАГРУЗКА ОБУЧЕННОЙ МОДЕЛИ И ТОКЕНИЗАТОРА

# Обязательно перед запуском теста!
model_dir = "/content/drive/MyDrive/finetuned_rubert_medical2_vkr"

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)

# Загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(
    model_dir,
    local_files_only=True
)

# Словарь названий классов
ID2LABEL = {
    0: "Гипотезы и открытия",
    1: "Дети",
    2: "Заболевания и лечение",
    3: "Здравоохранение и экспертные мнения",
    4: "Общественное здоровье",
    5: "Фармацевтика"
}

# Настройка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("Модель успешно загружена!")


Модель успешно загружена!


In [8]:
# Создание функции для проверки языка вводимого текста и его формата, а также отклонение при низкой уверенности.
def predict_with_rejection(text, threshold=0.45, max_length=256):

    # Проверка формата входных данных.
    if not isinstance(text, str) or not text.strip():
        return {"error": "Некорректный формат текста."}

    # Определение языка входных данных.
    # Если не удалось определить язык вводимого текста, то отклонить классификацию.
    try:
        lang = detect(text)
    except LangDetectException:
        return {
            "decision": "Отклонено",
            "reason": "Не удалось определить язык текста."
        }
    # Если текст не на русском языке, то отклонить классификацию.
    if lang != "ru":
        return {
            "decision": "Отклонено",
            "reason": "Текст не на русском языке.",
            "detected_language": lang
        }

    # Токенизация текста.
    enc = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Перенос на устройство.
    enc = {k: v.to(device) for k, v in enc.items()}

    # Предсказание модели
    with torch.no_grad():
        outputs = model(**enc)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    # Анализ уверенности
    max_idx = int(probs.argmax())
    max_prob = float(probs[max_idx])
    # Если уверенность низкая, то отклонить классификацию.
    if max_prob < threshold:
        return {
            "Решение": "Отклонено",
            "Причина": "Низкая уверенность модели",
            "Уверенность": round(max_prob, 4)
        }
    # Результаты классификации.
    return {
        "decision": "Классифицировано",
        "label_id": max_idx,
        "label_name": ID2LABEL[max_idx],
        "confidence": round(max_prob, 4),
        "detected_language": lang
    }

In [12]:
# ТЕСТ TPI-2 (После установки в новое окружение Smoke Test)
# Тестирование медицинского текста на русском языке - Дети.
print(predict_with_rejection(
    "Резкое переохлаждение может привести к потере сознания и остановке сердца, рассказала заведующая детским приемным отделением клиники Санкт-Петербургского государственного педиатрического медицинского университета Минздрава России Ксана Штернлихт"
))
# Тестирование медицинского текста на русском языке - Заболевания и лечение.
print(predict_with_rejection(
    "Для снижения вероятности развития деменции необходимо следить за давлением, холестерином и качеством сна, советует врач-невролог Наталья Жарова."
))
# Тестирование медицинского текста на русском языке - Общественное здоровье.
print(predict_with_rejection(
    "О ключевых признаках заболевания, ошибках лечения и симптомах, при которых опасно и стоит звать врача, рассказала врач-терапевт Надежда Чернышова "
))
# Тестирование медицинского текста на русском языке - Фармацевтика.
print(predict_with_rejection(
    "Аналоги лекарств дешевле оригинальных не потому, что они хуже, а потому что на их создание компания затрачивает меньше средств, рассказал РГ директор по связям с инвесторами компании Озон Фармацевтика Дмитрий Коваленко."
))
# Тестирование медицинского текста на русском языке - Гипотезы и открытия.
print(predict_with_rejection(
    "Участники исследования отметили улучшение субъективного ощущения сонливости, концентрации внимания и увеличение скорости реакции на 7-10% при ярком свете по сравнению с приглушенным освещением. "
))
# Тестирование медицинского текста на русском языке - Здравоохранение и экспертные мнения.
print(predict_with_rejection(
    "Минздрав: Просроченные лекарства вызывают потенциально смертельное заболевание"
))

# Тестирование медицинского текста на анлгийскойм языке.
print(predict_with_rejection(
    "The Ministry of Health approved new recommendations."
))
# Тестирование не правильного формата вводимых данных.
print(predict_with_rejection(
    "123_45?+32354_4239475243_59457766---3_2342==]1233445558_)))))_(((;;;)))"
))

{'decision': 'Классифицировано', 'label_id': 3, 'label_name': 'Здравоохранение и экспертные мнения', 'confidence': 0.7781, 'detected_language': 'ru'}
{'decision': 'Классифицировано', 'label_id': 2, 'label_name': 'Заболевания и лечение', 'confidence': 0.8752, 'detected_language': 'ru'}
{'decision': 'Классифицировано', 'label_id': 5, 'label_name': 'Фармацевтика', 'confidence': 0.7802, 'detected_language': 'ru'}
{'decision': 'Классифицировано', 'label_id': 5, 'label_name': 'Фармацевтика', 'confidence': 0.9934, 'detected_language': 'ru'}
{'decision': 'Классифицировано', 'label_id': 2, 'label_name': 'Заболевания и лечение', 'confidence': 0.9004, 'detected_language': 'ru'}
{'decision': 'Классифицировано', 'label_id': 5, 'label_name': 'Фармацевтика', 'confidence': 0.9864, 'detected_language': 'ru'}
{'decision': 'Отклонено', 'reason': 'Текст не на русском языке.', 'detected_language': 'en'}
{'decision': 'Отклонено', 'reason': 'Не удалось определить язык текста.'}
